# Downstream analysis
We create a vcf file containing small and structral variants annotated with their AF and predicted functional impact. Here we are going to go through how to explore those variants.

Let's start by visualizing structral variants to make sure that the callers did a good job.


I am going to


In [ ]:
%%bash 
grep "cuteSV-25-8240662-DEL-0-1405" results/cuteSV/ERR5043144.hifi.pbmm2.phased.vcf

# Samplot
The variant of length(1405) starts at 8240662 and ends at 8242067. We are going to use samplot to visualize this variant. 
Plotting using our snakemake workflow is very simple. 

just run 'snakemake -j1 -p results/samplot/{sv\_type}\_{chrom}\_{start}\_{end}.png'

In [ ]:
%%bash
snakemake -j1 -p results/samplot/DEL_25_8240662_8242067.png

In [ ]:
from IPython.display import Image
Image(filename='results/samplot/DEL_25_8240662_8242067.png') 

# Visualize SV Benchmark
Benchmarking SV is not an easy job because tools always disagree about the positions of breakpoints. Therefore, we can expect that some SV in our benchmarks tagged as FP while it was correct but the breakpoint wasnt matching.

Let's view a SV called by pbsv but was tagged as FP.


In [ ]:
%%bash
grep "pbsv-25-9733349-DEL-0-663" results/pbsv/ERR5043144.hifi.pbmm2.phased.vcf

In [ ]:
%%bash
snakemake -j1 -p results/samplot/DEL_25_9733349_9734012.png

In [ ]:
Image(filename='results/samplot/DEL_25_9733349_9734012.png') 

Looks like that SV is actually correct. Lesson here is that You have to visualize SV to make sure that everything is correct.

# Variant Effect predictor
We are going to use VEP to predict the effect of the variants. The following figgure summarizes the annotations produced by VEP. More information is available on their [website](https://uswest.ensembl.org/info/genome/variation/prediction/predicted_data.html)
![VEP](https://uswest.ensembl.org/info/genome/variation/prediction/consequences.jpg)

### Run VEP using snakemake
to get the output file for vep: replace the extnesion(".vcf.gz") of any compressed vcf file  with ".vep.vcf.gz".

for example: 

results/cuteSV/ERR7091271.ont.minimap2.phased.vcf.gz 

                will be

results/cuteSV/ERR7091271.ont.minimap2.phased.vep.vcf.gz 

In [ ]:
%%bash 
snakemake -j4 --use-conda "results/cuteSV/ERR7091271.ont.minimap2.phased.vep.vcf.gz"

### View VEP report
Lets first, look at the summary results they produced. 

1. Browse the folders using the panel on the left to "results/cuteSV/"

2. Download the report "ERR7091271.ont.minimap2.phased.vep.html": right click on the file then click downloand

### Let's visualize a high impact variant
We need first to get the coordinates of a high impact variant to visualize

In [ ]:
%%bash
zgrep "coding_sequence_variant"  results/cuteSV/ERR7091271.ont.minimap2.phased.vep.vcf.gz

Let's visualize the first deletion(396bp) starting from 2585287 to 2585683 on chromsome

In [ ]:
%%bash
snakemake -j1 -p results/samplot/DEL_25_2585287_2585683.png

In [ ]:
Image(filename='results/samplot/DEL_25_2585287_2585683.png')

## Population Frequency analysis
We calculated the AF for our VCFs in 10 samples. Here, I am providing a vcf file produced using the sample workflow but I ran the population genotyper against 428 samples. you will find the result file "final.vep.vcf.bgz" contianing all the vairants and "final.SV.vep.vcf.bgz" containing only the SV. The following table describes the metadata tagged for each variant.


| Metadata      | Description |
| -- |:-----------:|
| AC | Allele count in genotypes|
| AC_Het | Allele counts in homozygous genotypes|
| AC_Hom | Allele counts in heterozygous genotypes|
| AC_Hemi | Allele counts in hemizygous genotypes|
| AF | Allele frequency |
| MAF | Minor Allele frequency |
| NS | Number of samples with data   |
| AN | Total number of alleles in called genotypes |
| HWE | Hardy-Weinberg equilibrium |
| ExcHet | Test excess heterozygosity; 1=good, 0=bad |


Let's first check a file called "samples.csv" containing breed information of the 428 animal. The following command print the first 10 animals

In [ ]:
%%bash
cat samples.csv |head|tr -s ',' $'\t' | ../tools/prettytable 3 

The commands below count the number of samples per breed

In [ ]:
%%bash
cut -f2 -d, samples.csv |sort |uniq -c| awk '{print $2"\t"$1}' |sort -k2,2nr > tmp
cat <(echo -e "Breed\tcount") tmp | ../tools/prettytable 2

### Find Rare variants

Bcftools is very helpful in filtering vcf files using the variants metadata. For example, We can query the novel varaints using the following command

In [ ]:
%%bash
bcftools view  -Q 0.001 final.SV.vep.vcf.bgz  | grep -vP "^#"  |head -n 4

### Finding common variants
on the other hand we can select the most common variants

In [ ]:
%%bash
bcftools view  -q 0.9 final.SV.vep.vcf.bgz  | grep -vP "^#"  |head -n 4

# Hail
Although bcftools is very helpful and fast but it is hard to do complex tasks with it. Here we are suggesting using Hail to be able explore the population genotyping results and get meaningful results. Hail is a python library for genomic data expoloration. It creates a matrix table for vcf files which is very similar to R dataframes.

So let's do some coding by intializing Hail engine

In [ ]:
import hail as hl
hl.init()
from hail.plot import show
from pprint import pprint
hl.plot.output_notebook()

Now we are going to load the vcf and samples information to create Hail Matrix table

In [ ]:
ref="/home/mshokrof/workshop_12Jan_2023_data/ARS-UCD1.2_Btau5.0.1Y.25.fa"
index="/home/mshokrof/workshop_12Jan_2023_data/ARS-UCD1.2_Btau5.0.1Y.25.fa.fai"
vcf="final.SV.vep.vcf.bgz"
samplesInfo="samples.csv"
hlRef=hl.ReferenceGenome.from_fasta_file("ARSUCD",ref,index)

mt = hl.import_vcf(vcf,reference_genome=hlRef)
table = (hl.import_table('samples.csv', impute=True,delimiter=",")
         .key_by('BioSample'))
mt = mt.annotate_cols(breed = table[mt.s])

Lets see how the hail matrix table is organized

In [ ]:
mt.rows().show(5)

In [ ]:
mt.GT.show(5)

In [ ]:
samplesPercohort=mt.aggregate_cols(hl.agg.counter(mt.breed.Cohort))
print(samplesPercohort)

### Stratify population allele frequency
Here we are trying to answer questions like which variants are frequent in the Indicus breeds only. We are going to calculate allele frequencies per cohort.


In [ ]:
mt=mt.annotate_rows(AF_indicus=hl.agg.filter(mt.breed.Cohort =="indicus",
                                     hl.agg.sum(mt.GT.n_alt_alleles())
                                     / samplesPercohort["indicus"]*2 ))
mt=mt.annotate_rows(AF_taurus=hl.agg.filter(mt.breed.Cohort =="taurus",
                                     hl.agg.sum(mt.GT.n_alt_alleles())
                                     / samplesPercohort["taurus"]*2 ))
mt=mt.annotate_rows(AF_bosoutgroup=hl.agg.filter(mt.breed.Cohort =="bosoutgroup",
                                     hl.agg.sum(mt.GT.n_alt_alleles())
                                     / samplesPercohort["bosoutgroup"]*2 ))
mt.rows().show(5)

Now we calculated startified AF per cohort lets find the frequent variants in Indicus samples

In [ ]:
indicusFrequent=mt.filter_rows(mt.AF_indicus > 0.7)
indicusFrequent.rows().show()

We can easily get the ids of the common variants

In [ ]:
indicusFrequent.rows().rsid.collect()[:10]

Similarily we find common variants for the Holstein breed only.

In [ ]:
numSamples=mt.aggregate_cols(hl.agg.filter(mt.breed.CompositeBreed == "Holstein" ,hl.agg.count()))
mt=mt.annotate_rows(AF_Holstein=hl.agg.filter(mt.breed.CompositeBreed =="Holstein",
                                     hl.agg.sum(mt.GT.n_alt_alleles())
                                     / numSamples*2 ))

mt.filter_rows(mt.AF_Holstein > 0.8).rows().show()

## Explore population genotypes of a specfic variant

Let's explore the population data of the high impact variant that we visualized earlier


In [ ]:
HighImpactSV=mt.filter_rows(mt.rsid=="cuteSV-25-2585287-DEL-0-396")
HighImpactSV.rows().show()

In [ ]:
print("Indicus Freq =%.2f"%       HighImpactSV.rows().AF_indicus.collect()[0])
print("Taurus Freq =%.2f"%        HighImpactSV.rows().AF_taurus.collect()[0])
print("Bos out group Freq =%.2f"% HighImpactSV.rows().AF_bosoutgroup.collect()[0])

### Here we are showing the sum of alleles found per each breed.


In [ ]:
entries = HighImpactSV.entries()
results = (entries.group_by(breed = entries.breed.CompositeBreed)
      .aggregate(alleleCount = hl.agg.sum(entries.GT.n_alt_alleles())))
results=results.order_by(-results.alleleCount)
results.show()

Finally get the ids of the samples that have this variant

In [ ]:
entries = HighImpactSV.entries()
results = entries.filter(entries.GT.is_non_ref())
print(results.s.collect())

# Run principal component analysis (PCA) on the Hardy-Weinberg-normalized genotype call matrix.
Finally lets run pca on the genotypes and visualize how the samples are related to each others

In [ ]:
eigenvalues, pcs, _ = hl.hwe_normalized_pca(mt.GT)
mt = mt.annotate_cols(scores = pcs[mt.s].scores)


In [ ]:
from bokeh.models import  CategoricalColorMapper
from bokeh.palettes import Category10

pallete=Category10[3]
colors={
    'taurus': pallete[0],
    'indicus': pallete[1],
    'bosoutgroup': pallete[2]
    
}

colorTable={}
for s in table.collect():
    colorTable[s.CompositeBreed]=colors[s.Cohort]
factors=[]
pallete=[]
for k,v in colorTable.items():
    factors.append(k)
    pallete.append(v)
    
color_mapper = CategoricalColorMapper(factors=factors, palette=pallete)    

p = hl.plot.scatter(mt.scores[0],
                    mt.scores[1],
                    label=mt.breed.CompositeBreed,
                    colors=color_mapper,
                    title='PCA', xlabel='PC1', ylabel='PC2')
show(p)